In [32]:
import pandas as pd
import numpy as np
import datetime

In [33]:
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)

In [34]:
date_list = pd.read_excel('C:/Users/Infovesta PC-04/project_pegadaian/universe_creation/date_list.xlsx')
date_list = date_list.drop(columns='Unnamed: 0')
date_list = list(date_list['Date'])

In [35]:
bond_identity = pd.read_csv('C:/Users/Infovesta PC-04/project_pegadaian/universe_creation/bond_msbond.csv')
bond_price_1 = pd.read_csv('C:/Users/Infovesta PC-04/project_pegadaian/universe_creation/Bond_TrBei_to_20240816.csv')
bond_price_2 = pd.read_csv('C:/Users/Infovesta PC-04/project_pegadaian/universe_creation/Bond_TrBei_from_20240819_to_20250226.csv')
bond_price = pd.concat([bond_price_1,bond_price_2])
bond = bond_price.merge(bond_identity,on='portId',how='left')
bond['issuedyear'] = bond['issueddate'].str[:4]
bond['issuedyear'] = bond['issuedyear'].astype('int')

C:\Users\Infovesta PC-04\AppData\Local\Temp\ipykernel_17608\2125768073.py:2: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  bond_price_1 = pd.read_csv('C:/Users/Infovesta PC-04/project_pegadaian/universe_creation/Bond_TrBei_to_20240816.csv')


In [36]:
bond['portReleased'] = bond['portReleased'].replace(r'\N',np.nan)
bond['jtdatedt'] = pd.to_datetime(bond['jtdate'])
bond['portReleaseddt'] = pd.to_datetime(bond['portReleased'])
bond['tenorday'] = bond.apply(lambda row: np.busday_count(row['portReleaseddt'].date(), row['jtdatedt'].date()) if pd.notna(row['portReleaseddt']) and pd.notna(row['jtdatedt']) else np.nan, axis=1)
bond['tenoryear'] = bond['tenorday']/261 #asumsi 1 tahun ada 261 hari
bond = bond.drop(columns='jtdatedt')
bond = bond.drop(columns='portReleaseddt')

In [37]:
sbn = bond[bond['portType']=='S']
corpo = bond[bond['portType']=='B']
corpo

,txtNo,portId,portrank,portrank2,issueddate,highPrice,loPrice,lastPrice,frekuensi,volumeTrans,valueTrans,WAPPrice,coupon,YTMValue,MDValue,portId2,portName,portType,syariah,dollar,outstanding,jtdate,portissuer,portReleased,periodekupon,flag,issuedyear,tenorday,tenoryear
0,13036,ADHI04,idA-,A,2010-06-15,99.92,99.92,99.92,2,6.000,5.995400,99.9233,11.000,11.03845793,0.039611368,ADHI04,Obligasi IV Adhi Tahun 2007,B,N,N,3.750000e+11,2012-07-06,ADHI,2007-07-09,4.0,1.0,2010,1304.0,4.996169
1,13037,APEX02B,idA+,A,2010-06-15,100.00,100.00,100.00,2,6.000,6.000000,100.0000,15.000,14.99900874,0.016376051,APEX02B,Apexindo Pratama Duta II Tahun 2009 Seri B,B,N,N,3.000000e+11,2014-06-19,APEX,2009-06-22,4.0,1.0,2010,1303.0,4.992337
2,13038,APOL02A,idCCC,NIG,2010-06-15,97.51,96.15,96.15,17,45.520,43.884200,96.4064,12.000,13.58310608,0.01864385,APOL02A,Obligasi APOL II Tahun 2008 Seri A,B,N,N,2.760000e+11,2021-06-30,APOL,2008-03-19,4.0,1.0,2010,3465.0,13.275862
3,13039,ASDF11F,idAA-,AA,2010-06-15,102.00,101.70,101.93,4,12.000,12.227400,101.8950,10.900,10.28533161,0.02955736,ASDF11F,Obligasi Astra Sedaya Finance XI Tahun 2010 Se...,B,N,N,3.700000e+11,2014-03-18,ASDF,2010-03-19,4.0,1.0,2010,1042.0,3.992337
4,13040,BBNI01XXBFTW,AA(idn),AA,2010-06-15,105.05,104.90,104.90,2,4.000,4.199000,104.9750,13.125,8.211633492,0.06176981,BBNI01XXBFTW,Bank BNI I Tahun 2003,B,N,N,1.000000e+12,2011-07-10,BBNI,NaN,4.0,1.0,2010,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321945,569478,TPIA04BCN2,idAA-,AA,2025-02-26,103.50,103.50,103.50,1,0.100,0.103500,103.5000,8.900,8.044209,3.95318,TPIA04BCN2,Obligasi Berkelanjutan IV Chandra Asri Petroch...,B,N,N,2.273500e+11,2030-02-28,TPIA,2023-03-01,4.0,1.0,2025,1826.0,6.996169
321946,569479,MEDC05CCN1,idAA-,AA,2025-02-26,98.80,98.80,98.80,1,0.200,0.197600,98.8000,8.150,8.42917,4.253542,MEDC05CCN1,Obligasi Berkelanjutan V Medco Energi Internat...,B,N,N,5.900000e+11,2030-07-07,MEDC,2023-07-10,4.0,1.0,2025,1825.0,6.992337
321956,569489,PTRO01DCN1,idA+,A,2025-02-26,103.71,103.71,103.71,1,5.000,5.185500,103.7100,9.500,8.768894,4.919105,PTRO01DCN1,Obligasi Berkelanjutan I Petrosea Tahap I Tahu...,B,N,N,3.159600e+11,2031-12-13,PTRO,2024-12-16,4.0,1.0,2025,1825.0,6.992337
321962,569495,SPSMFBTN07A,idAAA(sf),AAA,2025-02-26,100.00,100.00,100.00,2,0.010,0.010000,100.0000,7.700,7.699287,6.316173,SPSMFBTN07A,EBA-SP SMF-BTN07 KELAS A,B,N,N,4.525000e+11,2034-01-07,SMFP,2022-12-23,4.0,1.0,2025,2881.0,11.038314


In [38]:
period = '3 Month'
one_month = 21 #days
rolling_dic = {'1 Month':one_month,'3 Month':one_month*3,'6 Month':one_month*6,'1 Year':one_month*12}
sbn_j_list = []
for en,i in enumerate(set(list(sbn['portId']))):
    # print(en)
    sbn_i = sbn[sbn['portId']==i].drop_duplicates()
    sbn_i = sbn_i.sort_values(by='issueddate')
    sbn_i['issueddatedt'] = pd.to_datetime(sbn_i['issueddate'])
    for j in list(sbn_i['issueddate'].unique()):
        sbn_j = sbn_i[sbn_i['issueddate']==j]
        try:
            cd_pos = date_list.index(j)
            pd_pos = cd_pos - rolling_dic[period]
            cd_ = datetime.datetime.strptime(j, '%Y-%m-%d')
            pd_ = datetime.datetime.strptime(date_list[pd_pos], '%Y-%m-%d')
            sbn_i_inside_dates = sbn_i[(sbn_i['issueddatedt']>=pd_)&(sbn_i['issueddatedt']<=cd_)]
            if len(sbn_i_inside_dates)>0:
                sbn_j[f'sumvalueTrans {period}'] = sum(sbn_i_inside_dates['valueTrans'])
                sbn_j[f'avgvalueTrans {period}'] = sum(sbn_i_inside_dates['valueTrans'])/len(sbn_i_inside_dates['valueTrans'])
                #sbn_j[f'pctvalueTrans {period}'] = len(sbn_i_inside_dates['valueTrans'])/(rolling_dic[period]+2)
                sbn_j_list.append(sbn_j)
            else: pass
        except ValueError: pass
sbn_vt = pd.concat(sbn_j_list)
sbn_vt = sbn_vt.reset_index(drop=True)
sbn_vt = sbn_vt.drop(columns='issueddatedt')


In [39]:
# Tambahkan suku bunga Bank Indonesia
kurs_BI = pd.read_excel("kurs_BI.xls")
kurs_BI.columns = kurs_BI.loc[0]
kurs_BI = kurs_BI[1:].reset_index(drop=True)
kurs_BI['Tanggal'] = kurs_BI['Tanggal'].apply(lambda x: x.strftime('%Y-%m-%d'))
kurs_BI = kurs_BI.rename(columns={'Nilai': 'USDIDR', 'Tanggal': 'issueddate'})
sbn_vt = pd.merge(sbn_vt, kurs_BI[['issueddate', 'USDIDR']], on='issueddate', how='left')

In [88]:
bond_nominal = pd.read_csv('bond_msnominal.csv').drop('txtno',axis=1).rename(columns={'tanggal': 'issueddate'})

sbn_bond_list = []

for kode in bond_nominal['portid'].unique():
    bond_i = bond_nominal[bond_nominal['portid'] == kode][['issueddate', 'nominal']]
    sbn_i = sbn_vt[sbn_vt['portId'] == kode]

    sbn_bond_i = pd.concat([bond_i,sbn_i]).sort_values('issueddate')
    sbn_bond_i['nominal'] = sbn_bond_i['nominal'].ffill()
    sbn_bond_i = sbn_bond_i.dropna(subset='portId')
    sbn_bond_list.append(sbn_bond_i)

sbn_vt_df = pd.concat(sbn_bond_list)
sbn_vt_df

,issueddate,nominal,txtNo,portId,portrank,portrank2,highPrice,loPrice,lastPrice,frekuensi,volumeTrans,valueTrans,WAPPrice,coupon,YTMValue,MDValue,portId2,portName,portType,syariah,dollar,outstanding,jtdate,portissuer,portReleased,periodekupon,flag,issuedyear,tenorday,tenoryear,sumvalueTrans 3 Month,avgvalueTrans 3 Month,USDIDR,portid+date
49566,2009-05-18,2.890000e+12,16916.0,FR0021,-,-,107.00,107.00,107.00,0.0,1.000,1.070000,107.00,14.5,9.5987,1.329176305,FR0021,Obligasi Negara Th. 2002 Seri FR0021,S,N,N,2.614000e+12,2010-12-15,NKRI,NaN,2.0,1.0,2009.0,NaN,NaN,1.070000,1.070000,10480,FR00212009-05-18
49567,2009-06-10,2.890000e+12,16917.0,FR0021,-,-,108.51,108.30,108.51,0.0,65.000,70.505500,108.47,14.5,8.4252,1.278166653,FR0021,Obligasi Negara Th. 2002 Seri FR0021,S,N,N,2.614000e+12,2010-12-15,NKRI,NaN,2.0,1.0,2009.0,NaN,NaN,71.575500,35.787750,10034,FR00212009-06-10
49568,2009-06-12,2.890000e+12,16918.0,FR0021,-,-,108.15,108.15,108.15,0.0,60.000,64.890000,108.15,14.5,8.6246,1.271357611,FR0021,Obligasi Negara Th. 2002 Seri FR0021,S,N,N,2.614000e+12,2010-12-15,NKRI,NaN,2.0,1.0,2009.0,NaN,NaN,136.465500,45.488500,10105,FR00212009-06-12
49569,2009-06-16,2.890000e+12,16919.0,FR0021,-,-,108.15,108.10,108.15,0.0,40.000,43.252000,108.13,14.5,8.5968,1.344382504,FR0021,Obligasi Negara Th. 2002 Seri FR0021,S,N,N,2.614000e+12,2010-12-15,NKRI,NaN,2.0,1.0,2009.0,NaN,NaN,179.717500,44.929375,10190,FR00212009-06-16
49570,2009-07-01,2.890000e+12,16920.0,FR0021,-,-,108.60,108.60,108.60,0.0,50.000,54.300000,108.60,14.5,8.1111,1.30834193,FR0021,Obligasi Negara Th. 2002 Seri FR0021,S,N,N,2.614000e+12,2010-12-15,NKRI,NaN,2.0,1.0,2009.0,NaN,NaN,234.017500,46.803500,10255,FR00212009-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45411,2025-01-30,1.750000e+12,566798.0,SPNS13102025,-,-,95.80,95.80,95.80,4.0,116.372,111.484376,95.80,6.6,0.0,0.663664,SPNS13102025,Surat Perbendaharaan Negara Seri SPNS13102025,S,N,N,1.750000e+12,2025-10-13,NKRI,2025-01-17,0.0,1.0,2025.0,191.0,0.731801,211.484376,105.742188,16200,SPNS131020252025-01-30
87327,2025-02-07,2.000000e+12,567598.0,SPN12260205,NaN,NaN,100.00,100.00,100.00,100.0,100.000,100.000000,100.00,6.4,0.0,0.979438,SPN12260205,Surat Perbendaharaan Negara Seri SPN12260205,S,N,N,2.000000e+12,2026-02-05,NKRI,2025-02-07,0.0,1.0,2025.0,259.0,0.992337,100.000000,100.000000,16330,SPN122602052025-02-07
87328,2025-02-18,2.000000e+12,568536.0,SPN12260205,-,-,94.35,94.35,94.35,2.0,137.143,129.394420,94.35,6.4,0.0,0.920003,SPN12260205,Surat Perbendaharaan Negara Seri SPN12260205,S,N,N,2.000000e+12,2026-02-05,NKRI,2025-02-07,0.0,1.0,2025.0,259.0,0.992337,229.394420,114.697210,16208,SPN122602052025-02-18
85572,2025-02-14,5.000000e+11,568269.0,SPNS10112025,NaN,NaN,100.00,100.00,100.00,100.0,100.000,100.000000,100.00,6.3,0.0,0.71999,SPNS10112025,Surat Perbendaharaan Negara Seri SPNS10112025,S,N,N,5.000000e+11,2025-11-10,NKRI,2025-02-14,0.0,1.0,2025.0,191.0,0.731801,100.000000,100.000000,16365,SPNS101120252025-02-14


In [89]:
sbn_vt_df = sbn_vt_df.rename(columns={'nominal': 'outstanding new'})

sbn_vt_df['outstanding adj'] = np.where(sbn_vt_df['dollar'] == 'Y', sbn_vt_df['outstanding new'] * pd.to_numeric(sbn_vt_df['USDIDR']), sbn_vt_df['outstanding new'])
sbn_vt_df

,issueddate,outstanding new,txtNo,portId,portrank,portrank2,highPrice,loPrice,lastPrice,frekuensi,volumeTrans,valueTrans,WAPPrice,coupon,YTMValue,MDValue,portId2,portName,portType,syariah,dollar,outstanding,jtdate,portissuer,portReleased,periodekupon,flag,issuedyear,tenorday,tenoryear,sumvalueTrans 3 Month,avgvalueTrans 3 Month,USDIDR,portid+date,outstanding adj
49566,2009-05-18,2.890000e+12,16916.0,FR0021,-,-,107.00,107.00,107.00,0.0,1.000,1.070000,107.00,14.5,9.5987,1.329176305,FR0021,Obligasi Negara Th. 2002 Seri FR0021,S,N,N,2.614000e+12,2010-12-15,NKRI,NaN,2.0,1.0,2009.0,NaN,NaN,1.070000,1.070000,10480,FR00212009-05-18,2.890000e+12
49567,2009-06-10,2.890000e+12,16917.0,FR0021,-,-,108.51,108.30,108.51,0.0,65.000,70.505500,108.47,14.5,8.4252,1.278166653,FR0021,Obligasi Negara Th. 2002 Seri FR0021,S,N,N,2.614000e+12,2010-12-15,NKRI,NaN,2.0,1.0,2009.0,NaN,NaN,71.575500,35.787750,10034,FR00212009-06-10,2.890000e+12
49568,2009-06-12,2.890000e+12,16918.0,FR0021,-,-,108.15,108.15,108.15,0.0,60.000,64.890000,108.15,14.5,8.6246,1.271357611,FR0021,Obligasi Negara Th. 2002 Seri FR0021,S,N,N,2.614000e+12,2010-12-15,NKRI,NaN,2.0,1.0,2009.0,NaN,NaN,136.465500,45.488500,10105,FR00212009-06-12,2.890000e+12
49569,2009-06-16,2.890000e+12,16919.0,FR0021,-,-,108.15,108.10,108.15,0.0,40.000,43.252000,108.13,14.5,8.5968,1.344382504,FR0021,Obligasi Negara Th. 2002 Seri FR0021,S,N,N,2.614000e+12,2010-12-15,NKRI,NaN,2.0,1.0,2009.0,NaN,NaN,179.717500,44.929375,10190,FR00212009-06-16,2.890000e+12
49570,2009-07-01,2.890000e+12,16920.0,FR0021,-,-,108.60,108.60,108.60,0.0,50.000,54.300000,108.60,14.5,8.1111,1.30834193,FR0021,Obligasi Negara Th. 2002 Seri FR0021,S,N,N,2.614000e+12,2010-12-15,NKRI,NaN,2.0,1.0,2009.0,NaN,NaN,234.017500,46.803500,10255,FR00212009-07-01,2.890000e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45411,2025-01-30,1.750000e+12,566798.0,SPNS13102025,-,-,95.80,95.80,95.80,4.0,116.372,111.484376,95.80,6.6,0.0,0.663664,SPNS13102025,Surat Perbendaharaan Negara Seri SPNS13102025,S,N,N,1.750000e+12,2025-10-13,NKRI,2025-01-17,0.0,1.0,2025.0,191.0,0.731801,211.484376,105.742188,16200,SPNS131020252025-01-30,1.750000e+12
87327,2025-02-07,2.000000e+12,567598.0,SPN12260205,NaN,NaN,100.00,100.00,100.00,100.0,100.000,100.000000,100.00,6.4,0.0,0.979438,SPN12260205,Surat Perbendaharaan Negara Seri SPN12260205,S,N,N,2.000000e+12,2026-02-05,NKRI,2025-02-07,0.0,1.0,2025.0,259.0,0.992337,100.000000,100.000000,16330,SPN122602052025-02-07,2.000000e+12
87328,2025-02-18,2.000000e+12,568536.0,SPN12260205,-,-,94.35,94.35,94.35,2.0,137.143,129.394420,94.35,6.4,0.0,0.920003,SPN12260205,Surat Perbendaharaan Negara Seri SPN12260205,S,N,N,2.000000e+12,2026-02-05,NKRI,2025-02-07,0.0,1.0,2025.0,259.0,0.992337,229.394420,114.697210,16208,SPN122602052025-02-18,2.000000e+12
85572,2025-02-14,5.000000e+11,568269.0,SPNS10112025,NaN,NaN,100.00,100.00,100.00,100.0,100.000,100.000000,100.00,6.3,0.0,0.71999,SPNS10112025,Surat Perbendaharaan Negara Seri SPNS10112025,S,N,N,5.000000e+11,2025-11-10,NKRI,2025-02-14,0.0,1.0,2025.0,191.0,0.731801,100.000000,100.000000,16365,SPNS101120252025-02-14,5.000000e+11


In [90]:
reb_date = ['2025-02-03','2024-10-01','2024-07-01','2024-04-01']

In [92]:
# Market Value - Accrued Interest
# Pietro Veronesi - Fixed Income Securities_ Valuation, Risk, and Risk Management (2010, Wiley)
# Chapter 2.4, 2.4.4.2 Treasury Coupon Notes and Bonds page 51

with pd.ExcelWriter(f'obligasi_negara_tanpa_min_jatuh_tempo_0_MV_new.xlsx') as writer:
    for date_ in reb_date:
        sbn_1 = sbn_vt_df.copy()
        sbn_1['marketvalue'] = sbn_1['lastPrice']*sbn_1['outstanding adj'] # Dikurangin accrual
        sbn_1['date'] = date_
        sbn_1['date'] = pd.to_datetime(sbn_1['date'])
        sbn_1['jtdatedt'] = pd.to_datetime(sbn_1['jtdate'])
        sbn_1['issueddatedt'] = pd.to_datetime(sbn_1['issueddate'])
        sbn_1 = sbn_1[sbn_1['jtdatedt']>date_]
        sbn_1 = sbn_1[sbn_1['issueddatedt']<date_]
        cd_pos = date_list.index(date_)
        pd_pos = cd_pos - rolling_dic[period]
        cd_ = datetime.datetime.strptime(date_, '%Y-%m-%d')
        pd_ = datetime.datetime.strptime(date_list[pd_pos], '%Y-%m-%d')
        sbn_1 = sbn_1[(sbn_1['issueddatedt']>=pd_)&(sbn_1['issueddatedt']<=cd_)]
        
        # Market Value - Accrued Interest
        coupon = sbn_1['coupon'] / 100
        outstanding = sbn_1['outstanding adj']
        periodekupon = sbn_1['periodekupon']  # Periode Kupon per tahun
        tenorday = sbn_1['tenorday']  # Umur Obligasi
        jtdate = sbn_1['jtdatedt']  # Tanggal Maturity
        valid_condition = (tenorday > 0.0) & (periodekupon >= 1.0) & (tenorday < 1e10) & (periodekupon < 1e15) & (outstanding > 0.0)
        coupon_interval = tenorday / periodekupon
        coupon_interval[~valid_condition] = None  # Set NaN, -inf, inf to None
        last_coupon_date = jtdate - pd.to_timedelta(coupon_interval, unit='D')
        days_since_last_coupon = abs((pd.to_datetime('today') - last_coupon_date).dt.days)
        # print((coupon == 0).sum(), (outstanding == 0).sum(), (periodekupon == 0).sum(), (days_since_last_coupon == 0).sum(), (coupon_interval == 0).sum())
        sbn_1['accrued_interest'] = (coupon * outstanding / periodekupon) * (days_since_last_coupon / coupon_interval) # Calculate accrued interest
        sbn_1['marketvalue_accruedinterest'] = (sbn_1['lastPrice'] * sbn_1['outstanding adj']) - sbn_1['accrued_interest'] # Calculate market value

        sbn_1 = sbn_1[['portId', 'portName', 'date','issueddate',
                    'portReleased','jtdate','avgvalueTrans 3 Month','lastPrice',
                    'outstanding adj','marketvalue', 'marketvalue_accruedinterest', 'accrued_interest']]
        
        sbn_1 = sbn_1.drop_duplicates(subset='portId',keep='last')
        sbn_1 = sbn_1.dropna(subset='marketvalue')
        sbn_1 = sbn_1[sbn_1['marketvalue']!=0]
        sbn_1['MV Weight'] = sbn_1['marketvalue']/sum(sbn_1['marketvalue'])
        sbn_1 = sbn_1.dropna(subset='marketvalue_accruedinterest')
        sbn_1 = sbn_1[sbn_1['marketvalue_accruedinterest']!=0]
        sbn_1['MV - AI Weight'] = sbn_1['marketvalue_accruedinterest']/sum(sbn_1['marketvalue_accruedinterest'])
        sbn_1 = sbn_1.rename(columns={'portId':'ID','portName':'Nama Obligasi','date':'Rebalancing Date',
                                    'issueddate':'Last Transaction Date','portReleased':'Issuance Date',
                                    'jtdate':'Jatuh Tempo','outstanding adj':'Nilai Outstanding','avgvalueTrans 3 Month':'Average Transaction Value 3 Month',
                                    'lastPrice':'Last Price','marketvalue':'Market Value', 'marketvalue_accruedinterest': 'MV - AI', 'accrued_interest': 'AI'})
        sbn_1['Rebalancing Date'] = sbn_1['Rebalancing Date'].astype('str')
        sbn_1 = sbn_1.sort_values(by='Average Transaction Value 3 Month',ascending=False)
        # sbn_1 = sbn_1.head(50) #Ambil 50 bond
        sbn_1 = sbn_1.reset_index(drop=True)
        sbn_1.to_excel(writer, sheet_name=f'{date_}')

sbn_1

,ID,Nama Obligasi,Rebalancing Date,Last Transaction Date,Issuance Date,Jatuh Tempo,Average Transaction Value 3 Month,Last Price,Nilai Outstanding,Market Value,MV - AI,AI,MV Weight,MV - AI Weight
0,FR0100,Obligasi Negara Republik Indonesia Seri FR0100,2024-04-01,2024-03-28,2023-08-25,2034-02-15,2992.859208,102.50,9.284150e+13,9.516254e+15,9.512047e+15,4.206275e+12,0.018995,0.019238
1,FR0101,Obligasi Negara Republik Indonesia Seri FR0101,2024-04-01,2024-03-28,2023-11-03,2029-04-15,2301.441246,101.35,8.878590e+13,8.998451e+15,8.995195e+15,3.256056e+12,0.017962,0.018193
2,FR0096,Obligasi Negara Republik Indonesia Seri FR0096,2024-04-01,2024-03-28,2022-08-22,2033-02-15,1182.188566,101.50,1.525640e+14,1.548525e+16,1.547938e+16,5.866688e+12,0.030910,0.031307
3,FR0102,Obligasi Negara Republik Indonesia Seri FR0102,2024-04-01,2024-03-28,2024-01-08,2054-07-15,1041.514342,99.26,2.155800e+13,2.139847e+15,2.138598e+15,1.249055e+12,0.004271,0.004325
4,FR0098,Obligasi Negara Republik Indonesia Seri FR0098,2024-04-01,2024-03-28,2022-09-16,2038-06-15,998.119434,103.50,8.152900e+13,8.438252e+15,8.434344e+15,3.907988e+12,0.016844,0.017058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,SR018T5,SUKUK NEGARA RITEL SERI SR018T5,2024-04-01,2024-03-28,2023-04-06,2028-03-10,3.432447,100.85,4.544820e+12,4.583451e+14,4.581764e+14,1.687309e+11,0.000915,0.000927
99,ORI024T3,Obligasi Negara Republik Indonesia Seri ORI024T3,2024-04-01,2024-03-28,2023-11-09,2026-10-15,2.852895,100.25,1.186280e+13,1.189246e+15,1.188780e+15,4.653939e+11,0.002374,0.002404
100,PBS007,SBSN Seri PBS007,2024-04-01,2024-03-15,2014-10-01,2040-09-15,2.249375,121.75,1.037500e+13,1.263156e+15,1.262846e+15,3.106066e+11,0.002521,0.002554
101,PBS023,SBSN Seri PBS023,2024-04-01,2024-03-18,2019-05-16,2030-05-15,2.195333,108.70,1.087500e+13,1.182112e+15,1.181980e+15,1.321233e+11,0.002360,0.002391


In [93]:
# Market Value - Accrued Interest (Excel IGBI 2025)

with pd.ExcelWriter(f'obligasi_negara_tanpa_min_jatuh_tempo_0_MV_new.xlsx') as writer:
    for date_ in reb_date:
        sbn_1 = sbn_vt_df.copy()
        sbn_1['marketvalue'] = sbn_1['lastPrice']*sbn_1['outstanding adj'] # Dikurangin accrual
        sbn_1['date'] = date_
        sbn_1['date'] = pd.to_datetime(sbn_1['date'])
        sbn_1['jtdatedt'] = pd.to_datetime(sbn_1['jtdate'])
        sbn_1['issueddatedt'] = pd.to_datetime(sbn_1['issueddate'])
        sbn_1 = sbn_1[sbn_1['jtdatedt']>date_]
        sbn_1 = sbn_1[sbn_1['issueddatedt']<date_]
        cd_pos = date_list.index(date_)
        pd_pos = cd_pos - rolling_dic[period]
        cd_ = datetime.datetime.strptime(date_, '%Y-%m-%d')
        pd_ = datetime.datetime.strptime(date_list[pd_pos], '%Y-%m-%d')
        sbn_1 = sbn_1[(sbn_1['issueddatedt']>=pd_)&(sbn_1['issueddatedt']<=cd_)]
        
        # Market Value - Accrued Interest
        coupon = sbn_1['coupon']/100
        outstanding = sbn_1['outstanding adj']
        ytm = sbn_1['YTMValue']/100 + 1
        valid_condition = (coupon > 0.0) & (outstanding > 0.0) & (ytm > 0.0)
        condition_check = coupon * ytm * outstanding
        condition_check[~valid_condition] = None  # Set NaN, -inf, inf to None
        
        # print((coupon == 0).sum(), (outstanding == 0).sum(), (ytm == 0).sum())
        sbn_1['accrued_interest'] = coupon * ytm * outstanding / 365 # Calculate accrued interest
        sbn_1['marketvalue_accruedinterest'] = (sbn_1['lastPrice'] * sbn_1['outstanding adj']) - sbn_1['accrued_interest'] # Calculate market value

        sbn_1 = sbn_1[['portId', 'portName', 'date','issueddate',
                    'portReleased','jtdate','avgvalueTrans 3 Month','lastPrice',
                    'outstanding adj','marketvalue', 'marketvalue_accruedinterest', 'accrued_interest']]
        
        sbn_1 = sbn_1.drop_duplicates(subset='portId',keep='last')
        sbn_1 = sbn_1.dropna(subset='marketvalue')
        sbn_1 = sbn_1[sbn_1['marketvalue']!=0]
        sbn_1['MV Weight'] = sbn_1['marketvalue']/sum(sbn_1['marketvalue'])
        sbn_1 = sbn_1.dropna(subset='marketvalue_accruedinterest')
        sbn_1 = sbn_1[sbn_1['marketvalue_accruedinterest']!=0]
        sbn_1['MV - AI Weight'] = sbn_1['marketvalue_accruedinterest']/sum(sbn_1['marketvalue_accruedinterest'])
        sbn_1 = sbn_1.rename(columns={'portId':'ID','portName':'Nama Obligasi','date':'Rebalancing Date',
                                    'issueddate':'Last Transaction Date','portReleased':'Issuance Date',
                                    'jtdate':'Jatuh Tempo','outstanding adj':'Nilai Outstanding','avgvalueTrans 3 Month':'Average Transaction Value 3 Month',
                                    'lastPrice':'Last Price','marketvalue':'Market Value', 'marketvalue_accruedinterest': 'MV - AI', 'accrued_interest': 'AI'})
        sbn_1['Rebalancing Date'] = sbn_1['Rebalancing Date'].astype('str')
        sbn_1 = sbn_1.sort_values(by='Average Transaction Value 3 Month',ascending=False)
        # sbn_1 = sbn_1.head(50) #Ambil 50 bond
        sbn_1 = sbn_1.reset_index(drop=True)
        sbn_1.to_excel(writer, sheet_name=f'{date_}')

sbn_1

,ID,Nama Obligasi,Rebalancing Date,Last Transaction Date,Issuance Date,Jatuh Tempo,Average Transaction Value 3 Month,Last Price,Nilai Outstanding,Market Value,MV - AI,AI,MV Weight,MV - AI Weight
0,FR0100,Obligasi Negara Republik Indonesia Seri FR0100,2024-04-01,2024-03-28,2023-08-25,2034-02-15,2992.859208,102.50,9.284150e+13,9.516254e+15,9516235840293062.0,17909706937.614765,0.018995,0.018995
1,FR0101,Obligasi Negara Republik Indonesia Seri FR0101,2024-04-01,2024-03-28,2023-11-03,2029-04-15,2301.441246,101.35,8.878590e+13,8.998451e+15,8998433145334459.0,17819665539.80875,0.017962,0.017962
2,FR0096,Obligasi Negara Republik Indonesia Seri FR0096,2024-04-01,2024-03-28,2022-08-22,2033-02-15,1182.188566,101.50,1.525640e+14,1.548525e+16,15485214760033290.0,31239966709.666611,0.030910,0.03091
3,FR0102,Obligasi Negara Republik Indonesia Seri FR0102,2024-04-01,2024-03-28,2024-01-08,2054-07-15,1041.514342,99.26,2.155800e+13,2.139847e+15,2139842737912387.0,4342087612.967409,0.004271,0.004271
4,FR0098,Obligasi Negara Republik Indonesia Seri FR0098,2024-04-01,2024-03-28,2022-09-16,2038-06-15,998.119434,103.50,8.152900e+13,8.438252e+15,8438234512915403.0,16987084596.741224,0.016844,0.016844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,SR018T5,SUKUK NEGARA RITEL SERI SR018T5,2024-04-01,2024-03-28,2023-04-06,2028-03-10,3.432447,100.85,4.544820e+12,4.583451e+14,458344251065591.875,845934408.133217,0.000915,0.000915
109,ORI024T3,Obligasi Negara Republik Indonesia Seri ORI024T3,2024-04-01,2024-03-28,2023-11-09,2026-10-15,2.852895,100.25,1.186280e+13,1.189246e+15,1189243598673848.5,2101326151.525274,0.002374,0.002374
110,PBS007,SBSN Seri PBS007,2024-04-01,2024-03-15,2014-10-01,2040-09-15,2.249375,121.75,1.037500e+13,1.263156e+15,1263153518117132.75,2731882867.170681,0.002521,0.002521
111,PBS023,SBSN Seri PBS023,2024-04-01,2024-03-18,2019-05-16,2030-05-15,2.195333,108.70,1.087500e+13,1.182112e+15,1182109924472869.5,2575527130.493774,0.002360,0.00236


In [ ]:
with pd.ExcelWriter(f'obligasi_negara_tanpa_min_jatuh_tempo_0.xlsx') as writer:
    for date_ in reb_date:
        sbn_1 = sbn_vt.copy()
        sbn_1['marketvalue'] = sbn_1['lastPrice']*sbn_1['outstanding'] # Dikurangin accrual
        # sbn_1['accruaedinterest'] = 
        sbn_1['date'] = date_
        sbn_1['date'] = pd.to_datetime(sbn_1['date'])
        sbn_1['jtdatedt'] = pd.to_datetime(sbn_1['jtdate'])
        sbn_1['issueddatedt'] = pd.to_datetime(sbn_1['issueddate'])
        sbn_1 = sbn_1[sbn_1['jtdatedt']>date_]
        sbn_1 = sbn_1[sbn_1['issueddatedt']<date_]
        cd_pos = date_list.index(date_)
        pd_pos = cd_pos - rolling_dic[period]
        cd_ = datetime.datetime.strptime(date_, '%Y-%m-%d')
        pd_ = datetime.datetime.strptime(date_list[pd_pos], '%Y-%m-%d')
        sbn_1 = sbn_1[(sbn_1['issueddatedt']>=pd_)&(sbn_1['issueddatedt']<=cd_)]
        sbn_1 = sbn_1[['portId','portName','date','issueddate','portReleased','jtdate','avgvalueTrans 3 Month','lastPrice','outstanding','marketvalue']]
        sbn_1 = sbn_1.drop_duplicates(subset='portId',keep='last')
        sbn_1 = sbn_1.dropna(subset='marketvalue')
        sbn_1 = sbn_1[sbn_1['marketvalue']!=0]
        sbn_1['Weight'] = sbn_1['marketvalue']/sum(sbn_1['marketvalue'])
        sbn_1 = sbn_1.rename(columns={'portId':'ID','portName':'Nama Obligasi','date':'Rebalancing Date','issueddate':'Last Transaction Date','portReleased':'Issuance Date','jtdate':'Jatuh Tempo','outstanding':'Nilai Outstanding','avgvalueTrans 3 Month':'Average Transaction Value 3 Month','lastPrice':'Last Price','marketvalue':'Market Value'})
        sbn_1['Rebalancing Date'] = sbn_1['Rebalancing Date'].astype('str')
        sbn_1 = sbn_1.sort_values(by='Average Transaction Value 3 Month',ascending=False)
        sbn_1 = sbn_1.head(50) #Ambil 50 bond
        sbn_1 = sbn_1.reset_index(drop=True)
        sbn_1.to_excel(writer, sheet_name=f'{date_}')